In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import pandas as pd

In [ ]:
review_ls = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/멀캠-프로젝트/스터디_프로젝트/review_project/olive_app_review_crawling_final.csv')

In [ ]:
review_ls.head()

,Unnamed: 0,날짜,별점,내용,추천
0,0,2022년 2월 7일,1,2/7 추가: 최신 버전 역시 여전히 느립니다. 피드백 반영도 안 하시면서 의견은 ...,238
1,1,2022년 2월 15일,1,"앱이 너무 느림, 리뷰달때 확인도 없이 바로 달림 길게써서 포인트 받을라 했는데 못...",0
2,2,2022년 2월 8일,1,진짜 별 하나도 아깝다.. 내가 내 돈주고 물건사려고 하는데 어플이 너무 느려서 장...,0
3,3,2022년 2월 8일,1,아 진짜 뭐하자는건지 모르겠네요 화나서 리뷰 쓰러 왔더니 많은 분들이 이미 지적을 ...,0
4,4,2022년 2월 4일,1,왜 별점이 2점대지..?싶을정도로 혜택도 많고 개좋은데 ㅜㅜㅜㅠ --->원래 별점 ...,11명


In [ ]:
df = pd.DataFrame(review_ls)
df['내용']

0       2/7 추가: 최신 버전 역시 여전히 느립니다. 피드백 반영도 안 하시면서 의견은 ...
1       앱이 너무 느림, 리뷰달때 확인도 없이 바로 달림 길게써서 포인트 받을라 했는데 못...
2       진짜 별 하나도 아깝다.. 내가 내 돈주고 물건사려고 하는데 어플이 너무 느려서 장...
3       아 진짜 뭐하자는건지 모르겠네요 화나서 리뷰 쓰러 왔더니 많은 분들이 이미 지적을 ...
4       왜 별점이 2점대지..?싶을정도로 혜택도 많고 개좋은데 ㅜㅜㅜㅠ --->원래 별점 ...
                              ...                        
2167                                                   최고
2168                                                  좋아요
2169                                                  좋아요
2170                                                   땡큐
2171                                                  좋아요
Name: 내용, Length: 2172, dtype: object

In [ ]:
docs = df['내용'].tolist()
docs

In [ ]:
n_topics = 4

tfidf_vect = TfidfVectorizer()
tfidf = tfidf_vect.fit_transform(docs)
svd = TruncatedSVD(n_components=n_topics, n_iter=100)

U = svd.fit_transform(tfidf)
Vt = svd.components_

In [ ]:
len(review_ls)

2172

In [ ]:
U.shape

(2172, 4)

In [ ]:
Vt.shape

(4, 11240)

In [ ]:
tfidf.shape

(2172, 11240)

In [ ]:
vocab =  tfidf_vect.get_feature_names_out()
print(len(vocab))

11240


In [ ]:
Vt

array([[ 5.47075985e-07,  3.09504441e-06,  3.32468230e-07, ...,
         1.42541136e-08,  3.59514518e-07,  2.79386083e-06],
       [ 7.28689737e-05,  1.74550742e-04,  1.08945402e-04, ...,
         2.09304534e-06,  1.80307416e-04,  3.03647697e-04],
       [ 5.05468132e-04,  8.42020161e-04,  1.40952472e-03, ...,
         9.86347551e-06,  6.90332309e-04,  9.26375875e-04],
       [-9.72211395e-05, -3.85788055e-04,  5.42449352e-04, ...,
        -1.55419314e-05, -3.58147743e-04, -5.71730529e-04]])

In [ ]:
n = 4
for i, topic in enumerate(Vt):
    print("Topic {} :".format(i), [ (vocab[i], topic[i].round(3)) for i in topic.argsort()[:-n-1:-1]])

Topic 0 : [('좋아요', 0.999), ('너무', 0.03), ('편하고', 0.014), ('완전', 0.011)]
Topic 1 : [('너무', 0.709), ('느려요', 0.639), ('앱이', 0.103), ('진짜', 0.085)]
Topic 2 : [('로그인이', 0.522), ('안됨', 0.388), ('로그인', 0.309), ('안돼요', 0.172)]
Topic 3 : [('느려요', 0.642), ('로그인이', 0.267), ('안됨', 0.21), ('로그인', 0.133)]


In [ ]:
n = 3
temp = [0, 1, 2, 3, 4, 5]
temp[:-n:-1]

[5, 4]

In [ ]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cosine_similarity(a, b):
    return dot(a, b) / (norm(a) * norm(b))

def calc_similarity_matrix(vectors):
    
    n_word = len(vectors)
    similarity_matrix = np.zeros((n_word, n_word))
    
    for i in range(n_word):
        for j in range(n_word):
            similarity_matrix[i, j] = cosine_similarity(vectors[i], vectors[j]).round(3)
            
    return similarity_matrix


In [ ]:
vectors = Vt.T
word_similarity_matrix = calc_similarity_matrix(vectors)
word_similarity_matrix.shape

(11240, 11240)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.rc("font", family="Malgun Gothic") #windows
# plt.rc("font", family="AppleGothic") #mac

def visualize_similarity(similarity_matrix, label):
    mask = np.triu(np.ones_like(similarity_matrix, dtype=np.bool))
    plt.rcParams['figure.figsize'] = [8, 6]
    ax = sns.heatmap(similarity_matrix, mask=mask, xticklabels=label, yticklabels=label, 
                     annot=True, fmt=".2f", annot_kws={"size":8}, cmap="coolwarm") 

In [ ]:
visualize_similarity(word_similarity_matrix, vocab)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  
findfont: Font family ['Malgun Gothic'] not found. Falling back to DejaVu Sans.
/usr/local/lib/python3.7/dist-packages/matplotlib/backends/backend_agg.py:214: RuntimeWarning: Glyph 50896 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/usr/local/lib/python3.7/dist-packages/matplotlib/backends/backend_agg.py:214: RuntimeWarning: Glyph 48512 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/usr/local/lib/python3.7/dist-packages/matplotlib/backends/backend_agg.py:214: RuntimeWarning: Glyph 53552 missing from current font.
  font.s

In [ ]:
doc_similarity_matrix = calc_similarity_matrix(U)
visualize_similarity(doc_similarity_matrix, review_ls)

In [ ]:
from sklearn.manifold import TSNE

def visualize_vectors(vectors, labels):
    tsne = TSNE(n_components=2, n_iter=10000, perplexity=2)
    T = tsne.fit_transform(vectors)
    
    plt.figure(figsize=(10, 6))
    plt.scatter(T[:, 0], T[:, 1], c='orange', edgecolors='r')
    for label, x, y in zip(labels, T[:, 0], T[:, 1]):
        plt.annotate(label, xy=(x+1, y+1), xytext=(0, 0), textcoords="offset points")


In [ ]:
visualize_vectors(vectors, vocab)

In [ ]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

import pandas as pd
from sklearn.datasets import fetch_20newsgroups


dataset = fetch_20newsgroups(shuffle=True, random_state=42, remove=("headers", "footer", "quotes"))


In [ ]:
!pip install pyLDAvis

In [ ]:
dataset['data'][:5]

In [ ]:
def get_news(news_dataset, apply_split=True):
    documents = news_dataset.data
    news_df = pd.DataFrame({"document": documents})
    news_df["clean_doc"] = news_df["document"].str.replace("[^a-zA-Z]", " ")
    news_df["clean_doc"] = news_df["clean_doc"].apply(lambda x : " ".join([w.lower() for w in x.split() if len(w) > 2]))
    tokenized_doc = news_df["clean_doc"].apply(lambda x: x.split())
    
    stop_words = stopwords.words('english')
    
    if apply_split:
        return tokenized_doc.apply(lambda x : [item for item in x if item not in stop_words])
    else:
        return tokenized_doc.apply(lambda x : " ".join([item for item in x if item not in stop_words]))
                                                      
    

In [ ]:
tokenized_docs = get_news(dataset, False)

In [ ]:
tokenized_docs[0]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

tfidf_vect = TfidfVectorizer()
tfidf = tfidf_vect.fit_transform(tokenized_docs)
lda = LatentDirichletAllocation(n_components=5, max_iter=50, learning_method="online", random_state=42)
lda_output = lda.fit_transform(tfidf)

In [ ]:
import pyLDAvis
import pyLDAvis.sklearn

pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda, tfidf, tfidf_vect, mds="tsne")
pyLDAvis.display(vis)